In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

import seaborn as sns

In [2]:
# Custom modules

import sys
import os
ROOT_DIR = os.path.abspath('../')
sys.path.insert(0, ROOT_DIR + "/src")

# Constant

In [3]:
RAW_INPUT_PATH = '../data/raw/bs140513_032310.csv'
IGNORE_COLS = ['zipcodeOri', 'zipMerchant']

In [22]:
TEST_RATIO = 0.2

In [62]:
OUTPUT_DIR = '../data/process/'

# Load input 

In [4]:
raw_df = pd.read_csv(RAW_INPUT_PATH,
                     quotechar="'",
                     usecols=lambda c: c not in IGNORE_COLS)

# Clean

## Drop noise (based on `01_eda` notebook)

### In label `gender`

In [8]:
clean_df = raw_df.query('gender in ("F", "M")')

### In label `age`

In [9]:
clean_df = clean_df.query('age != "U"')

# Divide train-test by customer

In [20]:
customer_list = clean_df['customer'].unique()

In [39]:
num_train_customer = round((1 - TEST_RATIO) * len(customer_list))
train_customer = np.random.choice(customer_list,
                                  size=num_train_customer,
                                  replace=False)
test_customer = set(customer_list) - set(train_customer)
test_customer = np.array(list(test_customer))

In [42]:
train_customer.shape

(3280,)

In [43]:
test_customer.shape

(820,)

In [ ]:
train_df = clean_df.loc[clean_df['customer'].isin(train_customer)]
test_df = clean_df.loc[clean_df['customer'].isin(test_customer)]

In [47]:
train_df.shape

(475657, 8)

In [48]:
test_df.shape

(117293, 8)

# Agg transformation per customer

- Perform quick aggregation per user to fit a baseline
- Ignore detail information about merchant and categories

### Continuous Features

#### Aggregate

In [64]:
agg_ops = {'step': ['count', 'mean', 'median'],
           'age': ['first'],
           'merchant': ['nunique'],
           'category': ['nunique'],
           'amount': ['mean', 'median', 'max', 'min', 'std'],
           'fraud': ['sum', 'mean'],
           'gender': ['first']}

In [65]:
train_agg_df = train_df.groupby('customer').agg(agg_ops)
test_agg_df = test_df.groupby('customer').agg(agg_ops)

#### Flatten column names

In [66]:
train_agg_df.columns = train_agg_df.columns.map('_'.join).str.strip()
test_agg_df.columns = test_agg_df.columns.map('_'.join).str.strip()

### Categorical features

In [102]:
CATE_AGG_DICT = {'age': 'first',
                 'merchant': 'count',
                 'category': 'count'}
CATE_FEATURES = list(CATE_AGG_DICT.keys())

#### Using pandas

In [113]:
def pd_one_hot(df, cate_agg_dict):
    pivot_df_list = []
    for col, aggfn in cate_agg_dict.items():
        pivot_df = pd.pivot_table(df, index=['customer'],
                                  columns=col, values='step',
                                  aggfunc=aggfn, fill_value=0)
        pivot_df.columns = pivot_df.columns.map(lambda s: f"{col}_{s}")
        pivot_df_list.append(pivot_df)
    cate_df = pd.concat(pivot_df_list, axis=1)
    return cate_df

In [114]:
train_cate_df = pd_one_hot(train_df, CATE_AGG_DICT)

In [116]:
test_cate_df = pd_one_hot(test_df, CATE_AGG_DICT)

# Join feature types

In [118]:
train_feature_df = pd.concat([train_agg_df, train_cate_df], axis=1)

In [120]:
test_feature_df = pd.concat([test_agg_df, test_cate_df], axis=1)

In [122]:
train_agg_df

,step_count,step_mean,step_median,age_first,merchant_nunique,category_nunique,amount_mean,amount_median,amount_max,amount_min,amount_std,fraud_sum,fraud_mean,gender_first
customer,,,,,,,,,,,,,,
C1000148617,131,107.786260,106.0,5,14,10,35.091908,26.14,323.64,0.80,36.863536,1,0.007634,M
C100045114,109,124.532110,134.0,4,22,11,90.336239,35.13,3902.93,1.37,381.812802,5,0.045872,M
C1000699316,94,51.436170,49.5,4,10,7,38.154894,28.32,326.34,0.99,41.544414,0,0.000000,M
C1001065306,30,91.833333,94.5,1,11,6,204.754667,103.68,1260.94,1.05,306.240251,11,0.366667,M
C1002658784,131,107.702290,109.0,3,17,12,34.253282,29.94,197.30,0.59,27.820062,0,0.000000,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C99729647,175,89.891429,91.0,3,14,9,32.555771,26.63,241.58,0.17,30.641718,0,0.000000,M
C998690782,171,89.005848,89.0,2,11,10,61.211637,26.22,5527.37,0.41,421.233339,2,0.011696,M
C998987490,177,90.107345,91.0,2,8,6,32.273107,26.20,219.99,0.54,30.122381,0,0.000000,F


# Persist

In [121]:
train_feature_df.to_csv(OUTPUT_DIR + "/train_feature.csv", index=False)
test_feature_df.to_csv(OUTPUT_DIR + "/test_feature.csv", index=False)

# Archive

#### Using sklearn.OneHotEncoder

In [75]:
from sklearn.preprocessing import OneHotEncoder

In [87]:
ohe = OneHotEncoder(sparse=False,
                    handle_unknown='ignore')

In [88]:
ohe.fit(train_df[CATE_FEATURES])

OneHotEncoder(handle_unknown='ignore', sparse=False)

In [92]:
train_cate_df = pd.DataFrame(ohe.transform(train_df[CATE_FEATURES]),
                             columns=ohe.get_feature_names(CATE_FEATURES))

In [93]:
train_cate_df

,age_0,age_1,age_2,age_3,age_4,age_5,age_6,merchant_M1053599405,merchant_M117188757,merchant_M1198415165,...,category_es_home,category_es_hotelservices,category_es_hyper,category_es_leisure,category_es_otherservices,category_es_sportsandtoys,category_es_tech,category_es_transportation,category_es_travel,category_es_wellnessandbeauty
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475652,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
475653,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
475654,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
475655,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [94]:
test_cate_df = pd.DataFrame(ohe.transform(test_df[CATE_FEATURES]),
                            columns=ohe.get_feature_names(CATE_FEATURES))

In [95]:
test_cate_df

,age_0,age_1,age_2,age_3,age_4,age_5,age_6,merchant_M1053599405,merchant_M117188757,merchant_M1198415165,...,category_es_home,category_es_hotelservices,category_es_hyper,category_es_leisure,category_es_otherservices,category_es_sportsandtoys,category_es_tech,category_es_transportation,category_es_travel,category_es_wellnessandbeauty
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117288,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
117289,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
117290,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
117291,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
